<a href="https://colab.research.google.com/github/AndreaHernandez5072/CienciaDatos/blob/main/HT9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Andrea Marleny Hernández Contreras, 1203718

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report,confusion_matrix,r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
#svm -> suport virtual machine
from sklearn.svm import SVC,LinearSVC
from sklearn.multiclass import OneVsOneClassifier,OneVsRestClassifier

In [ ]:
df = pd.read_csv("orange.csv")
df.info()

In [ ]:
df.isna().sum()

In [ ]:
print("Colors: ",df["Color"].unique())
print("\nVariety:",df["Variety"].unique())
print("\nBlemishes",df["Blemishes"].unique())

In [ ]:
cleanup={"Color":{"Orange":0,"Deep Orange":1,"Light Orange":2,"Orange-Red":3,"Yellow-Orange":4},
         "Variety":{"Valencia":0,"Navel":1,"Cara Cara":2,"Blood Orange":3,"Hamlin":4,"Tangelo (Hybrid)":5,"Murcott (Hybrid)":6,"Moro (Blood)":7,"Jaffa":8,
                    "Clementine":9,"Washington Navel":10,"Star Ruby":11,"Tangerine":12,"Ambiance":13,"California Valencia":14,"Honey Tangerine":15,"Navel (Late Season)":16,
                    "Clementine (Seedless)":17,"Temple":18,"Minneola (Hybrid)":19,"Satsuma Mandarin":20,"Midsweet (Hybrid)":21,"Navel (Early Season)":22,"Ortanique (Hybrid)":23},
         "Blemishes":{"N":0,"N (Minor)":1,"N (Split Skin)":2,"Y (Minor)":3,"Y (Sunburn)":4,"Y (Mold Spot)":5,"Y (Bruise)":6,"Y (Split Skin)":7,"Y (Sunburn Patch)":8,"Y (Scars)":9,
                      "Y (Minor Insect Damage)":10,"Y (Bruising)":11}}

df = df.replace(cleanup)
df.describe()

In [ ]:
#Detección de Outliers
def mostrarOutliers(dataset):
  fig,axs= plt.subplots(12,1,dpi=300,figsize=(7,30))
  i = 0
  for col in dataset.columns:
    axs[i].boxplot(df[col],vert=False)
    axs[i].set_ylabel(col)
    i+=1
  plt.show()

mostrarOutliers(df)

Variables que necesitan tratamiento son: Quality

In [ ]:
def TratamientoOutliers(col,dataset,cleandata):
  #calcular los cuartiles
  q1,q3=np.percentile(dataset[col],[25,75])
  #Calcular iqr
  iqr=q3-q1
  limite_inferior=q1-(1.5*iqr)
  limite_superior=q3+(1.5*iqr)
  #Similar a un IF
  dataset[col]=np.where(dataset[col]>limite_superior,limite_superior,np.where(dataset[col]<limite_inferior,limite_inferior,dataset[col]))

In [ ]:
clean_data=df
TratamientoOutliers("Quality",clean_data,clean_data)
clean_data.describe()

In [ ]:
mostrarOutliers(df)

In [ ]:
df.to_csv("out.csv",index=False)

In [ ]:
def identificarCorrelacion(dataset):
  f = dataset.corr()
  mask=np.triu(np.ones_like(f))
  #Ajustando el tamaño para que se vea mejor
  plt.figure(figsize=[10,7])
  sns.heatmap(f,annot=True,cmap="coolwarm",mask=mask,fmt=".2f",square=True, linewidths=.5)
  plt.title("Matriz de Correlación")
  plt.show()

identificarCorrelacion(clean_data)

In [ ]:
clean_data.plot.scatter(x="Brix",y="Quality")
plt.show()

# Separando data en
Usará Brix (nivel de dulzura), HarvestTime (días de cosecha), weight, softness y size

In [ ]:
data = clean_data

x = data.drop(["Quality"],axis=1)
y=data["Quality"]

x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=101,test_size=0.2)

# Regresion lineal

In [ ]:
modeloLR=LinearRegression().fit(x_train,y_train)
y_hatLR=modeloLR.predict(x_test)

In [ ]:
#Obtener la ecuación de la recta
print("Ecuación de la recta: y = (",modeloLR.coef_[0],") x + (",modeloLR.intercept_,")")
#Evaluando el modelo
print(confusion_matrix(round(y_test),y_hatLR.round()))
print("\n")
print(classification_report(y_test.round(),y_hatLR.round()))

# SVM


In [ ]:
svmmodel=SVC(kernel='linear',C=1000,gamma="auto",random_state=0)
#Entrenamiento del modelo
svmmodel.fit(x_train.round(),y_train.round())
#Proabndo las prediciones
y_hat=svmmodel.predict(x_test)
y_hat
#Evaluar el modelo
print(confusion_matrix(y_test.round(),y_hat.round()))
print("\n")
print(classification_report(y_test.round(),y_hat.round()))

# OnevsOneClassifier

In [ ]:
svmodel=OneVsOneClassifier(LinearSVC(C=10,random_state=1)).fit(x_train.round(),y_train.round())
y_hat=svmodel.predict(x_test)
#Evaluar el modelo
print(confusion_matrix(y_test.round(),y_hat.round()))
print("\n")
print(classification_report(y_test.round(),y_hat.round()))

# OneVsRestClassifier

In [ ]:
svmodel=OneVsRestClassifier(LinearSVC(C=10,random_state=0)).fit(x_train.round(),y_train.round())
y_hat=svmodel.predict(x_test)
#Evaluar el modelo
print(confusion_matrix(y_test.round(),y_hat.round()))
print("\n")
print(classification_report(y_test.round(),y_hat.round()))